In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import PIL
from PIL import Image

from torch.utils.data.dataset import Dataset
from torchvision import transforms

import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [5]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


8518


In [9]:
blacks = 0
whites = 0

In [10]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [41]:
blacks, whites

(1019906, 921217)

In [57]:
dataBlack = np.empty(blacks, dtype = '<U718')
dataWhite = np.empty(whites, dtype = '<U718')

In [58]:
dataBlack

array(['', '', '', ..., '', '', ''], dtype='<U718')

In [59]:
curBlack = 0
curWhite = 0

In [60]:
data.dtype

dtype('<U718')

In [61]:
for i in range(data.shape[0]):
    if data[i][0] == 'b':
        dataBlack[curBlack] = data[i]
        curBlack += 1
    elif data[i][0] == 'w':
        dataWhite[curWhite] = data[i]
        curWhite += 1

In [64]:
np.savetxt('dataBlack.txt', dataBlack, delimiter='\n', fmt="%s")

In [65]:
np.savetxt('dataWhite.txt', dataWhite, delimiter='\n', fmt="%s")

In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [176]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

def toBoards(gameBad, color):
    game = gameBad.split()
    res = []
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, len(game)):
        turn = game[i]
        curBoard = np.zeros(shape=(3, 15, 15))
        if curPlayer == 1:
            blackBoard[toTurn(turn)] = 1
        else:
            whiteBoard[toTurn(turn)] = -1
        
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        if i % 2 == 1 and color == "black":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
        elif i % 2 == 0 and color == "white":
            curBoard[0,:] = playerBoard
            res.append(curBoard)

        curPlayer *= -1
        #printBoard(curBoard)
    return np.asarray(res)

In [177]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [178]:
res1 = toBoards(dataBlack[0], "black") 
res2 = toBoards(dataBlack[1], "black") 
res3 = toBoards(dataBlack[2], "black") 


print(dataBlack[0])

black h8 h9 j10 k10 g11 f10 k9 l8 h11 f11 j8 g10 j9 j7 h10 f12 k8 e12 l7


In [179]:
print(res1.shape, res2.shape, res3.shape)


(10, 3, 15, 15) (20, 3, 15, 15) (14, 3, 15, 15)


#### Black

In [195]:
fileSizeBlack = 1019906 // 103
print(fileSizeBlack)
fileSizeWhite = 921217 // 83
print(fileSizeWhite)

9902
11099


In [196]:
curFile = 0
for curFile in range(103):
    for gm in range(10):
        ans = np.ndarray(shape=(0, 3, 15, 15))
        for i in range(1000):
            res = toBoards(dataBlack[curFile * fileSizeBlack + gm * 1000 + i], "black")
            ans = np.concatenate((ans, res), axis=0)

        print(ans.shape)
        curStr = "poses/black" + str(curFile) + ".npy"
        np.save(curStr, ans)
        curFile += 1

    

(15333, 3, 15, 15)
(16159, 3, 15, 15)
(15364, 3, 15, 15)
(15654, 3, 15, 15)
(14582, 3, 15, 15)
(18489, 3, 15, 15)
(17571, 3, 15, 15)
(16682, 3, 15, 15)
(16283, 3, 15, 15)
(16496, 3, 15, 15)
(15978, 3, 15, 15)
(16233, 3, 15, 15)
(15030, 3, 15, 15)
(15503, 3, 15, 15)
(16951, 3, 15, 15)
(16363, 3, 15, 15)
(13656, 3, 15, 15)
(19398, 3, 15, 15)
(16271, 3, 15, 15)
(15477, 3, 15, 15)
(15561, 3, 15, 15)


KeyboardInterrupt: 

In [191]:
ans.shape[0] / 20

3442.95

In [66]:
class CustomDatasetFromGames(Dataset):
    def __init__(self, img_paths):
        # Read data
        self.data_info = np.concatenate([np.load(file) for file in img_paths])
        self.feature_arr = np.asarray(self.data_info[:, 0])
        self.label_arr = np.asarray(self.data_info[:, 1])

        # Calculate len
        self.data_len = (self.data_info).shape[0]
        
    def __getitem__(self, index):
        # Get ndarray from index
        feature = self.feature_arr[index]
        label = self.label_arr[index]        
        return (feature, single_image_label)

    def __len__(self):
        return self.data_len


In [67]:
datasetBlack = CustomDatasetFromGames('dataBlack.txt')
trainLoaderBlack = torch.utils.data.DataLoader(dataset=data, batch_size=32, shuffle=False)

OSError: Failed to interpret file 'dataBlack.txt' as a pickle